In [2]:
# Install required libraries
!pip install datasets transformers
!apt-get install git-lfs
!pip install huggingface_hub

     |████████████████████████████████| 325 kB 5.4 MB/s 
     |████████████████████████████████| 3.8 MB 41.7 MB/s 
     |████████████████████████████████| 1.1 MB 33.4 MB/s 
     |████████████████████████████████| 212 kB 49.2 MB/s 
     |████████████████████████████████| 67 kB 5.4 MB/s 
     |████████████████████████████████| 134 kB 48.5 MB/s 
     |████████████████████████████████| 127 kB 47.7 MB/s 
     |████████████████████████████████| 596 kB 46.7 MB/s 
     |████████████████████████████████| 6.5 MB 31.6 MB/s 
     |████████████████████████████████| 895 kB 46.6 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 144 kB 49.5 MB/s 
     |████████████████████████████████| 271 kB 49.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling

In [3]:
import torch
torch.cuda.is_available()

# Load data
from datasets import load_dataset
goemo = load_dataset("go_emotions")

# Create a smaller training dataset for faster training times
small_train_dataset = goemo["train"]
small_test_dataset = goemo["test"]
# sampling of only single length labels
small_train_dataset = small_train_dataset.filter(lambda example: len(example['labels'])==1)
small_test_dataset = small_test_dataset.filter(lambda example: len(example['labels'])==1)


No config specified, defaulting to: go_emotions/simplified


Generating train split:   0%|          | 0/43410 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5426 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5427 [00:00<?, ? examples/s]

Dataset go_emotions downloaded and prepared to /root/.cache/huggingface/datasets/go_emotions/simplified/0.0.0/2637cfdd4e64d30249c3ed2150fa2b9d279766bfcd6a809b9f085c61a90d776d. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/44 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

In [4]:
# Set BERT tokenizer
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Prepare the text inputs for the model
def preprocess_function(examples):
    return tokenizer(examples["text"],padding=True, truncation=True)

tokenized_train = small_train_dataset.map(preprocess_function)
tokenized_test = small_test_dataset.map(preprocess_function)

# Use data_collector to convert our samples to PyTorch tensors and concatenate them with the correct amount of padding
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


# Define BERT as our base model:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=28)


import numpy as np
from datasets import load_metric

metric = load_metric("accuracy", "f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)



# Define a new Trainer with all the objects we constructed so far
from transformers import TrainingArguments, Trainer

repo_name = "kaka"

training_args = TrainingArguments(
    output_dir=repo_name,
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    weight_decay=0.01,
    save_strategy="epoch",
    # push_to_hub=True,
    group_by_length=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)


# Train the model
trainer.train()

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

  0%|          | 0/36308 [00:00<?, ?ex/s]

...

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, text. If id, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 36308
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 18154


Step,Training Loss
500,2.585300
1000,2.246200
1500,1.858400
2000,1.848200
2500,1.799000
3000,1.675500
3500,1.735500
4000,1.657000
4500,1.617800
5000,1.570900


Saving model checkpoint to kaka/checkpoint-18154
Configuration saved in kaka/checkpoint-18154/config.json
Model weights saved in kaka/checkpoint-18154/pytorch_model.bin
tokenizer config file saved in kaka/checkpoint-18154/tokenizer_config.json
Special tokens file saved in kaka/checkpoint-18154/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=18154, training_loss=1.6034215204487474, metrics={'train_runtime': 2502.1195, 'train_samples_per_second': 14.511, 'train_steps_per_second': 7.255, 'total_flos': 356177219067360.0, 'train_loss': 1.6034215204487474, 'epoch': 1.0})

In [5]:
# Compute the evaluation metrics
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, text. If id, text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 4590
  Batch size = 1


{'epoch': 1.0,
 'eval_accuracy': 0.6311546840958606,
 'eval_loss': 1.4020460844039917,
 'eval_runtime': 78.9793,
 'eval_samples_per_second': 58.116,
 'eval_steps_per_second': 58.116}